In [1]:
# used to widen the cells 
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 2.5% !important; width:95%; }</style>"))


In [2]:
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction

algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = "fFG6R4JTFq9Hh8RjZy130aMRdUhmOYp68hi9nm4L"
headers = {
   "X-API-Key": algod_token,
}

algod_client = algod.AlgodClient(algod_token, algod_address, headers);


In [3]:
funded_mnemonics= ["rookie similar wire owner wash once fish mosquito glad coffee family venture adult funny melt gas inspire tuna buzz sell dignity pottery gold able bracket",
                  "narrow mimic suffer top suspect follow menu broccoli try snake meat erase napkin lucky client forget sense bread glad eight uniform bacon code about crack", 
                  "city truth device clog grocery sea safe slide glove borrow swap capable trash shaft vast start space distance calm wire hub crush dose ability army", 
                  "unknown arctic antenna country credit december ill practice lawsuit today athlete rescue exit swarm fitness strong minimum soldier wide coffee vacuum piece coil absorb unable", 
                  "gravity adult destroy demand margin coast culture base adjust east banana certain happy daughter bless fiscal fiscal eye forget inspire brain banner evil able bonus"]

funded_accounts = [{'sk': mnemonic.to_private_key(memo), 'pk': mnemonic.to_public_key(memo)} for memo in funded_mnemonics]

In [4]:
def compile(approval_filename='approval.teal', clear_state_filename='clear_state.teal'):

    # pyteal to teal 
    with open(approval_filename, 'w') as f:
        compiled = compileTeal(approval_program(), Mode.Application)
        f.write(compiled)

    with open(clear_state_filename, 'w') as f:
        compiled = compileTeal(clear_state_program(), Mode.Application)
        f.write(compiled)

    # teal to bytecode 
    stdout, stderr = execute(["goal", "clerk", "compile", "-o", approval_filename+'.tok', approval_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    stdout, stderr = execute(["goal", "clerk", "compile", "-o", clear_state_filename+'.tok', clear_state_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    with open(approval_filename+'.tok', 'rb') as f:
        approval_bytes = f.read()

    with open(clear_state_filename+'.tok', 'rb') as f: 
        clear_state_bytes = f.read() 
        
    return approval_bytes, clear_state_bytes

In [5]:
# read app global state
def read_global_state(client, addr, app_id):   
    results = client.account_info(addr)
    apps_created = results['created-apps']
    global_states = []
    for app in apps_created :
        if app['id'] == app_id :
            global_states.append(app['params']['global-state'])
            # print(f"global_state for app_id {app_id}: ", app['params']['global-state'])
    return global_states 

In [6]:
import base64 
def decode_keys(s):
    for entry in s[0]:
        entry['key'] = base64.b64decode(entry['key']).decode('utf-8')
    return s

In [17]:
from pyteal import *    
    
def approval_program():     
    VOTING_CREDIT_SYM = Bytes("QVoteDecisionCredits")
    OPTION_PREFIX = Bytes("option_")
    NULL_OPTION = Bytes("NULL_OPTION")
    ZERO = Int(2**63)     # to have neative numbers 
    MINUS = Bytes("-")
    
    arg_num = Txn.application_args.length()
    
    on_closeout = Return(Int(1))
    
    asset_id = App.globalGet(Bytes("asset_id"))
    asset_balance = AssetHolding.balance(Int(0), asset_id) 
    asset_coeff = App.globalGet(Bytes("asset_coefficient"))
    on_optin = Seq([     # TODO if not before voting start time 
        asset_balance, 
        If(asset_balance.hasValue(),  # if sender has some tokens, store it, otherwise return 0     
           Seq([App.localPut(Int(0), VOTING_CREDIT_SYM, Mul(asset_balance.value(), asset_coeff)), Return(Int(1))]),      
           Return(Int(0))
        ) 
    ])  
    
    option0 = Concat(OPTION_PREFIX, Txn.application_args[1])
    option0_tally = App.globalGetEx(Int(0), option0)            # current total of votes for this option
    option0_votes = Btoi(Txn.application_args[2])               # how much the user wants to vote for this option 
    option0_votes_sign = Bytes(Txn.application_args[3])
    credit_balance = App.localGet(Int(0), VOTING_CREDIT_SYM)
    on_vote = Seq([ 
        option0_tally, 
        If(option0_tally.hasValue(),      # if user chose a valid option 
           Seq([
               App.localPut(Int(0), VOTING_CREDIT_SYM, credit_balance-Mul(option0_votes, option0_votes)),     # update balance
               If(Ge(credit_balance, Int(0)),             # does credit_balance this get re-evaluated? 
                    If(option0_votes_sign == MINUS,
                       App.globalPut(option0, option0_tally.value()-option0_votes),    
                       App.globalPut(option0, option0_tally.value()+option0_votes), 
                      )
                    Return(Int(0))
                 ),    
               Return(Int(1))
           ]),
           Return(Int(0))
          )
    ])
    
    
    # add up to 5 options at a time. 
    on_add_options =  Seq([
        If(Txn.application_args[1] != NULL_OPTION, 
           Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[1]), ZERO)]),
           Return(Int(1))
        ),
        If(Txn.application_args[2] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[2]), ZERO)]),
           Return(Int(1))
        ),
        If(Txn.application_args[3] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[3]), ZERO)]),
           Return(Int(1))
        ),
        If(Txn.application_args[4] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[4]), ZERO)]),
           Return(Int(1))
        ),
        If(Txn.application_args[5] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[5]), ZERO)]),
           Return(Int(1))
        ), 
        Return(Int(1))
    ])
    
    
    on_creation = Seq([    
        App.globalPut(Bytes("Creator"), Txn.sender()),    
        App.globalPut(Bytes("Name"), Txn.application_args[0]),
        App.globalPut(Bytes("asset_id"), Btoi(Txn.application_args[6])),  # should be Int 
        App.globalPut(Bytes("asset_coefficient"), Btoi(Txn.application_args[7])), 
        on_add_options,   # record the options 
        Return(Int(1))
    ])    
    
        
    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))], 
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin],
            [Txn.application_args[0] == Bytes("vote"), on_vote],
            [Txn.application_args[0] == Bytes("add_options"), on_add_options]
    )    
    return program    
    
    
def clear_state_program():     
    program = Return(Int(1))
    return program    


## Deploy

In [18]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema
from algosdk.future.transaction import OnComplete as onComplete

In [33]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

In [19]:
asset_id = 13164495    # this asset is used to compare 
asset_coefficient = 2    # how many voting coins you get for each token of the asset you own 

approval_bytes, clear_state_bytes = compile()

decision_name = 'muchdecision'

local_schema = StateSchema(num_uints=1, num_byte_slices=1)    
global_schema = StateSchema(num_uints=61, num_byte_slices=3)     # maximum sum is 64

on_complete = onComplete(0)
app_create_txn = transaction.ApplicationCreateTxn(
    funded_accounts[4]['pk'], 
    params, 
    on_complete, 
    approval_bytes, 
    clear_state_bytes, 
    global_schema, 
    local_schema,
    # you always need to submit this many options. you can use NULL_OPTION to ingore an option 
    app_args = [decision_name.encode('utf-8'), 
                "first".encode('utf-8'),
                "second".encode('utf-8'),
                "third".encode('utf-8'), 
                "NULL_OPTION".encode('utf-8'),
                "NULL_OPTION".encode('utf-8'), 
                asset_id.to_bytes(3, 'big'),
                asset_coefficient.to_bytes(2, 'big')]
)

app_create_txn_signed = app_create_txn.sign(funded_accounts[4]['sk'])
txid = algod_client.send_transaction(app_create_txn_signed)
txid

'2LIZ5WCJJ3VFWZK7JP2KMD6TQ4LCKB4FNTDYVNREQ7ZCLZUYEEFQ'

In [20]:
app_id = algod_client.pending_transaction_info(txid)['application-index']
app_id

14707630

In [24]:
global_state = read_global_state(algod_client, funded_accounts[4]['pk'], app_id)
decode_keys(global_state)

[[{'key': 'option_first', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_second', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_third', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'Creator',
   'value': {'bytes': 'hmXzebxJfQ9OITtFlPuRxlG9X6Sb/YeOo4wVxBw0Bh0=',
    'type': 1,
    'uint': 0}},
  {'key': 'Name',
   'value': {'bytes': 'bXVjaGRlY2lzaW9u', 'type': 1, 'uint': 0}},
  {'key': 'asset_id', 'value': {'bytes': '', 'type': 2, 'uint': 13164495}},
  {'key': 'asset_coefficient', 'value': {'bytes': '', 'type': 2, 'uint': 2}}]]

## Add more options

In [25]:
def add_options_tx(option_names, sender=4):
    app_args = ["add_options".encode("utf-8")] + [option_name.encode('utf-8') for option_name in option_names]

    # create unsigned transaction
    call_tx = transaction.ApplicationNoOpTxn(funded_accounts[sender]['pk'], params, app_id, app_args)
    call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[sender]['sk']))
    return call_txid

call_txid = add_options_tx(["new_one", "new_two", "new_three", "new_four", "NULL_OPTION"])

In [27]:
global_state = decode_keys(read_global_state(algod_client, funded_accounts[4]['pk'], app_id))
global_state

[[{'key': 'asset_coefficient', 'value': {'bytes': '', 'type': 2, 'uint': 2}},
  {'key': 'option_first', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_new_one', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_new_two', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_second', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_third', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'Name',
   'value': {'bytes': 'bXVjaGRlY2lzaW9u', 'type': 1, 'uint': 0}},
  {'key': 'asset_id', 'value': {'bytes': '', 'type': 2, 'uint': 13164495}},
  {'key': 'option_new_three', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_new_four', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'Creator',
   'value': {'bytes': 'hmXzebxJfQ9OITtFlPuRxlG9X6Sb/YeOo4wVxBw0Bh0=',
    'type': 1,
    'uint': 0}}]]

## Opt-in

In [ ]:
optin_tx = transaction.ApplicationOptInTxn(funded_accounts[2]['pk'], params, app_id)
algod_client.send_transaction(optin_tx.sign(funded_accounts[2]['sk']))

In [34]:
optin_tx = transaction.ApplicationOptInTxn(funded_accounts[3]['pk'], params, app_id)
algod_client.send_transaction(optin_tx.sign(funded_accounts[3]['sk']))

optin_tx = transaction.ApplicationOptInTxn(funded_accounts[4]['pk'], params, app_id)
algod_client.send_transaction(optin_tx.sign(funded_accounts[4]['sk']))

'XQYWPXUGBSIPMEPQAFS6E3VVZ4V5LYNAH4JYLH53IHUMZUTJJPKA'

In [35]:
local_storage = algod_client.account_info(funded_accounts[2]['pk'])['apps-local-state']
local_storage

[{'id': 14637979,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 0}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14707630,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 146}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [37]:
local_storage = algod_client.account_info(funded_accounts[3]['pk'])['apps-local-state']
local_storage

[{'id': 14707630,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 128}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [38]:
local_storage = algod_client.account_info(funded_accounts[4]['pk'])['apps-local-state']
local_storage

[{'id': 14707630,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 186}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [41]:
# if we look at the asset balances of these accounts we can check that the voting credits are asset_coefficient * asset_balance 
# account 1 has already opted in too many apps, that's why it isn't here

In [40]:
for acc in funded_accounts:
    print(algod_client.account_info(acc['pk'])['assets'])

[]
[{'amount': 42, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]
[{'amount': 73, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]
[{'amount': 64, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]
[{'amount': 93, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]


In [42]:
# let's try opting in an account with no assets 
optin_tx = transaction.ApplicationOptInTxn(funded_accounts[0]['pk'], params, app_id)
algod_client.send_transaction(optin_tx.sign(funded_accounts[0]['sk']))
# this tx is rejected by the approval program 

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction N3XCGIKJPRNOCUXH3GS5Q5LNTNKWYRZA5UILGLH6XOCO4UIBQNIQ: transaction rejected by ApprovalProgram"}


## Voting

In [43]:
option_name = "first"
votes = 5
sign = '+'

app_args = ["vote".encode("utf-8"), option_name.encode("utf-8"), votes.to_bytes(2, "big"), sign.encode("utf-8")]

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(funded_accounts[2]['pk'], params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[2]['sk']))

In [44]:
# balance has gone down by the square of the amount 
local_storage = algod_client.account_info(funded_accounts[2]['pk'])['apps-local-state']
local_storage

[{'id': 14637979,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 0}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14707630,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 121}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [46]:
# vote tally has gone up by the right amount 
global_state = decode_keys(read_global_state(algod_client, funded_accounts[4]['pk'], app_id))
global_state

[[{'key': 'option_third', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_new_three', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_new_four', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'Name',
   'value': {'bytes': 'bXVjaGRlY2lzaW9u', 'type': 1, 'uint': 0}},
  {'key': 'asset_coefficient', 'value': {'bytes': '', 'type': 2, 'uint': 2}},
  {'key': 'option_new_one', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_new_two', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'option_second', 'value': {'bytes': '', 'type': 2, 'uint': 0}},
  {'key': 'Creator',
   'value': {'bytes': 'hmXzebxJfQ9OITtFlPuRxlG9X6Sb/YeOo4wVxBw0Bh0=',
    'type': 1,
    'uint': 0}},
  {'key': 'asset_id', 'value': {'bytes': '', 'type': 2, 'uint': 13164495}},
  {'key': 'option_first', 'value': {'bytes': '', 'type': 2, 'uint': 5}}]]

In [45]:
# TODO test negative votes
# TODO 2 decimal places 